In [95]:
import pandas as pd
%matplotlib inline
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
import numpy as np #importing various libraries from sklearn to be used in the program
iris = datasets.load_iris() #imported the iris dataset

In [96]:
MT=pd.read_csv("seeds.csv")

In [97]:
MT.head()

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [98]:
MT.isnull().sum()

Area               0
Perimeter          0
Compactness        0
Kernel.Length      0
Kernel.Width       0
Asymmetry.Coeff    0
Kernel.Groove      0
Type               0
dtype: int64

# Separating data and label

In [99]:
X = MT.drop('Type', axis=1) #data 
Y = MT['Type'] #label

X.shape,Y.shape

((199, 7), (199,))

# Splitting the train and test data

In [100]:
# Split feature and label sets to train and data sets.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((159, 7), (40, 7), (159,), (40,))

# Scaling the data

In [101]:
scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(X_train)
scaler.fit(X_test)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Decision Tree

In [102]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion="entropy",max_depth=5)
#tree = DecisionTreeClassifier()
tree.fit(X_train, Y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, Y_train)))

Accuracy on training set: 0.987


In [103]:
print("Accuracy on testing set: {:.3f}".format(tree.score(X_test, Y_test)))

Accuracy on testing set: 0.950


In [104]:
print("Feature importances:\n{}".format(tree.feature_importances_))

Feature importances:
[0.29446813 0.         0.         0.018332   0.02730554 0.0792968
 0.58059753]


# Random Forest DT

In [105]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=90, random_state=0)

forest.fit(X_train, Y_train)

print("Accuracy on training set: {:.3f}".format(forest.score(X_train, Y_train)))

print("Accuracy on test set: {:.3f}".format(forest.score(X_test, Y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.925


In [106]:
print("Feature importances:\n{}".format(forest.feature_importances_))

Feature importances:
[0.19129981 0.24314129 0.03570991 0.11723228 0.13734651 0.06388485
 0.21138535]


# Gradient Boosting

In [107]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=0,learning_rate=0.1)

gbrt.fit(X_train, Y_train)

print("Accuracy on training set: {:.3f}".format(gbrt.score(X_train, Y_train)))

print("Accuracy on test set: {:.3f}".format(gbrt.score(X_test, Y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.950


In [108]:
print("Feature importances:\n{}".format(gbrt.feature_importances_))

Feature importances:
[0.40964439 0.03700145 0.00133308 0.00656728 0.02141326 0.0863449
 0.43769563]


# Naive Bayes

In [109]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, Y_train)
gnb.score(X_train,Y_train)

0.9056603773584906

In [110]:
gnb.score(X_test,Y_test)

0.975

# Logistic Regression

In [111]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

In [112]:
logisticRegr.fit(X_train, Y_train)

LogisticRegression()

In [113]:
score = logisticRegr.score(X_train, Y_train)
print(score)

0.9371069182389937


In [114]:
score = logisticRegr.score(X_test, Y_test)
print(score)

0.975


# SVM

In [115]:
clf = svm.SVC()
clf.fit(X_train,Y_train)


print('Accuracy of SVC on test set: {:.2f}'.format(clf.score(X_test, Y_test) * 100))
print('Accuracy of SVC on training set: {:.2f}'.format(clf.score(X_train, Y_train) * 100))


Accuracy of SVC on test set: 97.50
Accuracy of SVC on training set: 94.97


# Pipeline

In [116]:
from sklearn.metrics import classification_report #imporitng classification report

# With Logistic Regression

In [117]:
#Python scikit-learn provides a Pipeline utility to help automate machine learning workflows.
#Pipelines work by allowing for a linear sequence of data transforms to be chained together 
#culminating in a modeling process that can be evaluated.

pipe = Pipeline([('classifier' , RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['lbfgs','liblinear','sag','saga']},
    {'classifier': [GaussianNB()]},
    {'classifier': [GradientBoostingClassifier(random_state=0.1,learning_rate=200)],
    'classifier__n_estimators':list(range(10,101,10))},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10))},
    {'classifier' : [svm.SVC(C=0.1,gamma=1,kernel='rbf')]}] #feeding logistic regression, random forest, naive bayesian and support vector in the pipeline
    

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1) #cross validation

# Fit on data

best_clf = clf.fit(X_train, Y_train) #best classification on both train and test data



Fitting 2 folds for each of 182 candidates, totalling 364 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 238 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 364 out of 364 | elapsed:    2.6s finished
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [118]:
best_clf = clf.fit(X_train, Y_train) #best classification on both train and test data

best_clf.best_estimator_.get_params()['classifier'] #identifying the best classifier which is coming out to be random forest

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 182 candidates, totalling 364 fits


[Parallel(n_jobs=-1)]: Done 116 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 364 out of 364 | elapsed:    3.0s finished
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(C=1.623776739188721, penalty='l1', solver='saga')

In [119]:
grid_predictions = best_clf.predict(X_test) 

In [120]:
print('Model accuracy is',best_clf.score(X_test, Y_test)) #accuracy of the model which pipeline selected

Model accuracy is 1.0


Logistic regression is the best classifier

# Without Logistic Regression

In [121]:
#Python scikit-learn provides a Pipeline utility to help automate machine learning workflows.
#Pipelines work by allowing for a linear sequence of data transforms to be chained together 
#culminating in a modeling process that can be evaluated.

pipe = Pipeline([('classifier' , RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier': [GaussianNB()]},
    {'classifier': [GradientBoostingClassifier(random_state=0.1,learning_rate=200)],
    'classifier__n_estimators':list(range(10,101,10))},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10))},
    {'classifier' : [svm.SVC(C=0.1,gamma=1,kernel='rbf')]}] #feeding logistic regression, random forest, naive bayesian and support vector in the pipeline
    

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1) #cross validation

# Fit on data

best_clf = clf.fit(X_train, Y_train) #best classification on both train and test data



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 22 candidates, totalling 44 fits


[Parallel(n_jobs=-1)]: Done  44 out of  44 | elapsed:    1.7s finished


In [122]:
best_clf = clf.fit(X_train, Y_train) #best classification on both train and test data

best_clf.best_estimator_.get_params()['classifier'] #identifying the best classifier which is coming out to be random forest

Fitting 2 folds for each of 22 candidates, totalling 44 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 out of  44 | elapsed:    1.4s finished


SVC(C=0.1, gamma=1)

In [123]:
grid_predictions = best_clf.predict(X_test) 

In [124]:
print('Model accuracy is',best_clf.score(X_test, Y_test)) #accuracy of the model which pipeline selected

Model accuracy is 0.975


SVC is the best classifier